In [ ]:
from bspump.jupyter import *
import bspump.http.web.source
import bspump.file
import time
import json
import base64
from jinja2 import Environment, FileSystemLoader
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
new_pipeline("WhatsappMapsPipeline")

In [ ]:
@register_source
def source(app, pipeline):
   return bspump.http.web.source.WebHookSource(
       app,
       pipeline
   )

In [ ]:
sample_events([json.dumps(d) for d in [[
  {"time":"8:00 AM", "lat": 51.505, "lon": -0.09, "name": "Main St & First Ave"},
  {"time":"9:00 AM", "lat": 51.515, "lon": -0.10, "name": "Second St & Third Ave"},
  {"time":"10:00 AM", "lat": 51.525, "lon": -0.11, "name": "Fourth St & Fifth Ave"}
]]])

In [ ]:
@step
def parse_json(event):
    return json.loads(event)

In [ ]:
@step
def render_tempalte(event):
    env = Environment(loader=FileSystemLoader('.'))
    template_path = "/opt/pipelines/examples/Jupyter/timetable.html"
    if not os.path.isfile(template_path):
        template_path = "./timetable.html"
    template = env.get_template('timetable.html')
    return template.render(items=event)

In [ ]:
def setup_chrome_mobile_emulation():
    chrome_options = Options()
    chrome_options.add_argument("--headless")  # Run in headless mode
    chrome_options.add_argument("--no-sandbox")  # Bypass OS security model
    chrome_options.add_argument("--disable-gpu")  # Applicable to windows os only
    chrome_options.add_argument("--disable-dev-shm-usage")  # Overcome limited resource problems
    return webdriver.Chrome(options=chrome_options)

@step
def render_html_content(html_content):
    driver = setup_chrome_mobile_emulation()
    # Encode HTML content to base64
    encoded_content = base64.b64encode(html_content.encode('utf-8')).decode('utf-8')
    # Load the content using a data URL
    driver.get(f"data:text/html;base64,{encoded_content}")
    content_height = driver.execute_script("return document.body.scrollHeight")
    driver.set_window_size(360, content_height+70)  # Adjust window to content height
    time.sleep(0.2)
    screenshot = driver.get_screenshot_as_png()  # Get screenshot as a binary blob
    driver.quit()
    if is_running_in_jupyter():
        from IPython.display import Image, display
        display(Image(screenshot))
    return screenshot

In [ ]:
@register_sink
def init_sink(app, pipeline):
    return bspump.file.FileCSVSink(app, pipeline)

In [ ]:
end_pipeline()